In [1]:

# =============================================================================
# CONSUMPTION 19 MALAWI VILLAGE
# =============================================================================
import numpy as np
import pandas as pd
import os 
os.chdir('C:/Users/rodri/Dropbox/Chied_Field_June_19/Data/Consumption')
percentiles = [0.05, 0.1, .25, .5, .75, 0.8, 0.9, 0.95, 0.99]
euro_MWK = 806.64

# =============================================================================
# Import data: Data from the field and conversion rates (ISA-LSMS price conversions)
# =============================================================================

data = pd.read_stata("[2] Consumption+Non-Food Consumption_(9-16)_050819.dta", convert_categoricals=False)

##### Create conversion kg matrix(unitxitems) with the exact same names and units labels as in the data:


#item labels data
list_items = ['maizemgaiwa', 'maizerefined', 'maizemadeya', 'maizegrain', 'greenmaize', 'rice', 'cassavatubers', 'wsweetpotatoes', 'osweetpotatoes'
, 'ipotatoes', 'potatocrisps', 'bbean', 'pigeonpea', 'groundnut', 'groundnutf', 'onion', 'cabbage', 'tanaposi', 'leafyvegetables', 'tomato', 'eggs', 
'driedfish', 'fleshfish', 'goat', 'chicken', 'otherpoultry', 'smockedfish', 'mango', 'banana', 'guava', 
'wildfruits',  'sugar', 'sugarcane', 'cookingoil', 'softdrinks',
'thobwa', 'locallybrewed', 'salt']

list_items.append('fingermillet')
list_items.append('mandazidou')
# As it stands today (7/16/19) no one consumed otherpoultry, mango. That means no variables with such label. Need to drop:
#list_items.remove('otherpoultry')

#list_items.remove('mango')



'''
# NOW I ALREADY CREATED IT THE CONVERSION FILE SO NO NEED TO RUN IT AGAIN ===============
data_conversion = pd.read_excel('conversionkgrates_consumption_malawi_short.xls')

data_conversion = pd.read_stata('ConversionFactorsphysical.dta')
data_conversion = data_conversion.loc[data_conversion.region=='South',['hh_g02', 'hh_g03b','ihps_plus']]

items_conv = data_conversion.item_label.unique()
# Item labels conversion file
list_items_conv = ['Maize Mgaiwa', 'Maize refined', 'Maize Madeya', 'Maize grain',
       'Greenmaize', 'Rice', 'Cassava tubers',
       'White sweet potatoes', 'Orange sweet potatoes', 'Irish Potatoe',
       'Potatoe Chips', 'Bean, brown', 'Pigeon peas (ndolo)', 'Groundnut',
       'Groundnut flour', 'Onion', 'Cabbage', 'Tanaposi/Rape',
       'Other cultivated green leafy vegetables', 'Tomatoe', 'Eggs',
       'Dried Fish', 'Fresh Fish', 'Goat', 'Chicken',
       'Other poultry-guinea fowl, doves, etc', 'Smoked Fish', 'Mango',
       'Banana', 'Guava', 'Wild fruit (Masau, Malambe, etc)', 'Sugar',
       'Sugar Cane', 'Cooking oil',
       'Soft drinks(coca-cola, fanta, sprite etc)', 'Thobwa',
       'Locally brewed liquor(kachasu)', 'Salt']




conversionkg = data_conversion[['unit','item_label','conversion_kgs_country']]


# Reshape as: rows:units, columns:crops
conversionkg = conversionkg.replace(list_items_conv, list_items)
conversionkg_pivot = conversionkg.pivot_table(values='conversion_kgs_country',
                                index='unit',
                                columns='item_label')
conversionkg_pivot.loc[99,:] = np.nan
conversionkg_pivot.loc[100,:] = np.nan
conversionkg_pivot.to_csv('conversionkg_isaprices_matrix.csv')
'''

conversionkg_pivot = pd.read_csv('conversionkg_isaprices_matrix.csv',  index_col=0)

conversionkg_pivot.rename(columns={'Finger Millet':'fingermillet'}, inplace=True)
conversionkg_pivot.rename(columns={'Other poultry':'otherpoultry'}, inplace=True)
conversionkg_pivot['mandazidou'] = np.nan

conversionkg_pivot.to_csv('conversionkg_isaprices_matrix.csv',  index=True)

# =============================================================================
# Generate empty variables
# =============================================================================

#Obtain the names of the variables per each question of item. Question c is monetary question so not conversion to kgs needed
a_var = []
b_var = []
c_var = []
d_var = []

#Generate variable lables in a list
for item in list_items :
    a = item+'_a'
    b= item+'_b'
    c = item+'_c'  ## expenditure 
    d = item+'_d'
   
    a_var.append(a)
    b_var.append(b)
    c_var.append(c)
    d_var.append(d)
    
list_questions = ['a','b','d']

 
# convert all empty observations to 0. I do that to convert empty units to 99. If not, main conversion loop doesn't work.
# Note that empty doesn't necessary mean 0, so we careful at looking the data       
data = data.stack().apply(pd.to_numeric, errors='ignore').fillna(0).unstack()



# Drop nan observations. Also drop unit 25 (number not in our choices). Also drop 24(whole chicken). Drop empty cells observations
print('UNITS SELECTED PER ITEM')



#Find the households-questions that reported other units.
df_other_units = pd.DataFrame(columns=['householdid','question', 'other_unit']) 
for var in list_items:
    for i in range(1,4):   #Loop over unit questions.       
        # Find who said other units
        other_units_guy = data.loc[data[var+'_unit'+str(i)]=='other', ['householdid', var+'_unit'+str(i), var+'_unit'+str(i)+'_other']]            
        if other_units_guy.empty:
            continue
        else:
            d = {'householdid': other_units_guy.iloc[:,0], 'question': other_units_guy.columns[1], 'other_unit':other_units_guy.iloc[:,2]}   
            row = pd.DataFrame(data=d)
            df_other_units = df_other_units.append(row)
        
df_other_units.to_csv('other_units_list.csv')
print('All households-item-question combinations that reported "other" units')         
print(df_other_units)


for var in list_items:
    for i in range(1,4):   #Loop over unit questions.
        data[[var+'_unit'+str(i)]] = data[[var+'_unit'+str(i)]].replace('other', 99)
        data[[var+'_unit'+str(i)]] = data[[var+'_unit'+str(i)]].replace(np.nan, 99)
        data[[var+'_unit'+str(i)]] = data[[var+'_unit'+str(i)]].replace(25, 99)
        data[[var+'_unit'+str(i)]] = data[[var+'_unit'+str(i)]].replace(24, 99)
        data[[var+'_unit'+str(i)]] = data[[var+'_unit'+str(i)]].replace(0, 99)

#%% =============================================================================
#   Convert to kgs:  
# =============================================================================

# Generate kg variables empty
for item in list_items:
    for q in list_questions:
        data[item+'_'+q+'kg']= np.nan
        

print('a: Total Consumption')
for var in a_var:
    item = var[:-2]
    for i in range(len(data)):
        data.iloc[i,data.columns.get_loc(var+'kg')] = data.iloc[i,data.columns.get_loc(var)]*conversionkg_pivot.loc[int(data.iloc[i,  data.columns.get_loc(item+'_unit1')]), item]
    print(data[[var+'kg']].describe())


UNITS SELECTED PER ITEM
All households-item-question combinations that reported "other" units
    householdid              question other_unit
58           68     maizemgaiwa_unit1        5kg
58           68     maizemgaiwa_unit2        5kg
163         189      maizegrain_unit1    Handful
203         236      maizegrain_unit1    Handful
197         230  wsweetpotatoes_unit1       Pots
..          ...                   ...        ...
74           85   locallybrewed_unit2       25ml
16           21            salt_unit1     Packet
166         192            salt_unit1    Handful
16           21            salt_unit2     Packet
166         192            salt_unit2    Handful

[81 rows x 3 columns]
a: Total Consumption
       maizemgaiwa_akg
count       160.000000
mean         10.808362
std           9.633957
min           0.058824
25%           5.000000
50%          10.000000
75%          15.555555
max          70.000000
       maizerefined_akg
count        144.000000
mean           8.78

In [2]:

print('b: Bought')
for var in b_var:
    item = var[:-2]
    for i in range(len(data)): 
        data.iloc[i,data.columns.get_loc(var+'kg')] = data.iloc[i,data.columns.get_loc(var)]*conversionkg_pivot.loc[int(data.iloc[i,  data.columns.get_loc(item+'_unit2')]), item]

print('d: Own-produced')
for var in d_var:
    item = var[:-2]
    for i in range(len(data)): 
        data.iloc[i,data.columns.get_loc(var+'kg')] = data.iloc[i,data.columns.get_loc(var)]*conversionkg_pivot.loc[int(data.iloc[i,  data.columns.get_loc(item+'_unit3')]), item]



b: Bought
d: Own-produced


In [3]:
#%%  CONVERT TO MONETARY VALUE

# Generate price variables
for item in list_items:
        data[item+'_price']= np.nan

list_items.remove('mango')
list_items.remove('fingermillet')
# price per household
for item in list_items:
    data[item+'_price'] = data[item+'_c'] / data[item+'_bkg'].replace(0,np.nan)

price_data = pd.DataFrame(list_items, columns=['good'])
price_data['p_c'] = np.nan

for item in list_items:
    print('Median Price 1 kg of '+item)
    data['med_price_'+item] = data[item+'_price'].median()
    print(data['med_price_'+item].mean())    
    price_data.loc[price_data['good']==item,'p_c'] = data['med_price_'+item].mean()

price_data.to_csv('village_c_prices.csv', index=False)
## compute consumption in MWK
    
for item in list_items:
    for q in list_questions:
        data[item+'_'+q+'MWK']= np.nan

print('a: Total Consumption')
for item in list_items:
    print(item)
    data[item+'_aMWK'] = data[item+'_akg']*data['med_price_'+item]
    print('Food Consumption in MWK during last 7 days item: '+item)
    print(data[item+'_aMWK'].describe(percentiles=percentiles))
            
print('b: Bought')
for item in list_items:
    print(item)
    data[item+'_bMWK'] = data[item+'_bkg']*data['med_price_'+item]


print('d: Own-produced')
for item in list_items:
    #print(item)
    data[item+'_dMWK'] = data[item+'_dkg']*data['med_price_'+item]
    
    
#check total consumption
data['c_food'] = 0
data['c_food_purch'] = 0
data['c_food_ownprod'] = 0

for item in list_items:
    data['c_food'] += data[item+'_aMWK'].replace(np.nan, 0)
    data['c_food_purch'] += data[item+'_bMWK'].replace(np.nan, 0)
    data['c_food_ownprod'] += data[item+'_dMWK'].replace(np.nan, 0)

data[['c_food', 'c_food_purch' , 'c_food_ownprod']] = data[['c_food', 'c_food_purch' , 'c_food_ownprod']].replace(0.0,np.nan)


sumcfood= ((data[['c_food', 'c_food_purch' , 'c_food_ownprod']]*4/euro_MWK).replace(0.0,np.nan)).describe(percentiles)  
print('==== Summary Food Consumption at Month level in Euros =======')
print(sumcfood) 

Median Price 1 kg of maizemgaiwa
180.0
Median Price 1 kg of maizerefined
155.0
Median Price 1 kg of maizemadeya
200.0
Median Price 1 kg of maizegrain
323.07691122653193
Median Price 1 kg of greenmaize
325.0003341588836
Median Price 1 kg of rice
580.0
Median Price 1 kg of cassavatubers
265.00035183923364
Median Price 1 kg of wsweetpotatoes
50.0
Median Price 1 kg of osweetpotatoes
142.85714528998523
Median Price 1 kg of ipotatoes
350.0
Median Price 1 kg of potatocrisps
16666.66600439281
Median Price 1 kg of bbean
1000.0
Median Price 1 kg of pigeonpea
250.0
Median Price 1 kg of groundnut
800.0
Median Price 1 kg of groundnutf
700.2800959100449
Median Price 1 kg of onion
900.0089854899057
Median Price 1 kg of cabbage
200.0
Median Price 1 kg of tanaposi
5.0
Median Price 1 kg of leafyvegetables
631.552327504059
Median Price 1 kg of tomato
160.71492650478473
Median Price 1 kg of eggs
90.90908893868973
Median Price 1 kg of driedfish
533.3333333333317
Median Price 1 kg of fleshfish
960.038407908

C:\Users\rodri\Anaconda3\lib\site-packages\numpy\lib\nanfunctions.py:1018: RuntimeWarning: Mean of empty slice
  return np.nanmean(a, axis, out=out, keepdims=keepdims)


count      160.000000
mean      1945.505240
std       1734.112198
min         10.588235
5%         180.000000
10%        184.537826
25%        900.000000
50%       1800.000000
75%       2799.999962
80%       2799.999962
90%       3600.000000
95%       5409.999996
99%       8399.999886
max      12600.000000
Name: maizemgaiwa_aMWK, dtype: float64
maizerefined
Food Consumption in MWK during last 7 days item: maizerefined
count     144.000000
mean     1361.420094
std      1233.239085
min       116.250000
5%        116.250000
10%       141.100939
25%       484.375000
50%      1291.666617
75%      1550.000000
80%      1937.500000
90%      3100.000000
95%      3231.750000
99%      5812.500000
max      9041.666322
Name: maizerefined_aMWK, dtype: float64
maizemadeya
Food Consumption in MWK during last 7 days item: maizemadeya
count     104.000000
mean      416.602565
std       495.024647
min       100.000000
5%        133.333337
10%       133.333337
25%       133.333337
50%       273.333335
75%

In [5]:
data['nonf_cons_a_1']

0       800
1      2500
2      5050
3      3700
4      2750
       ... 
247    3000
248    2500
249    1530
250     500
251    3200
Name: nonf_cons_a_1, Length: 252, dtype: object

In [6]:

data['c_housing'] = data['nonf_cons_a_1']
data['c_clothes'] = data['nonf_cons_b_1'] 
data['c_education'] = data['nonf_cons_c_1'] 
data['c_health'] = data['nonf_cons_d_1']
data['c_funeralout'] = data['nonf_cons_e_1']
data['c_funeralin'] = data['nonf_cons_f_1']
data['c_weddingout'] = data['nonf_cons_g_1']
data['c_weddingin'] = data['nonf_cons_h_1'] 

data['c_nonfood'] = data[['c_housing', 'c_clothes', 'c_education', 'c_health', 'c_funeralout', 'c_funeralin', 'c_weddingout', 'c_weddingin']].sum(axis = 1, skipna = True)

sum_cnonfood = ((data[['c_nonfood','c_housing', 'c_clothes', 'c_education', 'c_health', 'c_funeralout', 'c_funeralin', 'c_weddingout', 'c_weddingin']]/euro_MWK).replace(0.0,np.nan)).describe()
print('======== SUMMARY  NON-FOOD CONSUMPTION (MONTH LEVEL)')
print('summary in EUR')
sum_cnonfood

======== SUMMARY  NON-FOOD CONSUMPTION (MONTH LEVEL)
summary in EUR


,c_nonfood,c_housing,c_clothes,c_education,c_health,c_funeralout,c_funeralin,c_weddingout,c_weddingin
count,250.000000,249.000000,120.000000,113.000000,151.000000,39.000000,9.000000,22.000000,4.000000
mean,25.886279,16.803951,10.844036,2.881614,3.119555,1.004483,12.355643,1.467930,1.689105
std,158.069436,156.880446,29.357693,4.230200,6.082038,1.868309,18.673388,1.884914,0.954087
min,0.185957,0.123971,0.009918,0.099177,0.030993,0.061986,1.239710,0.247942,0.557870
25%,3.719131,1.859566,1.239710,0.619855,0.371913,0.123971,2.479421,0.619855,1.069250
50%,8.120103,4.400972,3.719131,1.487652,1.239710,0.309928,2.727363,0.619855,1.859566
75%,20.215648,9.297828,8.987900,3.719131,3.719131,1.239710,12.395864,1.239710,2.479421
max,2482.210156,2479.730735,226.371120,33.472181,60.125955,11.157394,59.506099,8.677973,2.479421


In [8]:
#%% SAVE DATASET 

# Export dataset
data['hhid'] = data['householdid']

## short dataset
datacon_short = data[['hhid','c_food','c_food_purch','c_food_ownprod', 'c_nonfood','c_housing', 'c_clothes', 'c_education', 'c_health', 'c_funeralout', 'c_funeralin', 'c_weddingout', 'c_weddingin']]

## Food at monthly level
datacon_short[['c_food','c_food_purch','c_food_ownprod']] = datacon_short[['c_food','c_food_purch','c_food_ownprod']]*4

## Consumption at rainy season
datacon_short[['c_food','c_food_purch','c_food_ownprod', 'c_nonfood','c_housing', 'c_clothes', 'c_education', 'c_health', 'c_funeralout', 'c_funeralin', 'c_weddingout', 'c_weddingin']] = datacon_short[['c_food','c_food_purch','c_food_ownprod', 'c_nonfood','c_housing', 'c_clothes', 'c_education', 'c_health', 'c_funeralout', 'c_funeralin', 'c_weddingout', 'c_weddingin']]*7


datacon_short.to_csv('cons_short_19.csv', index=False)


##Long dataset (not at rainy season)
data.to_csv('cons_long_19.csv', index=False)


C:\Users\rodri\Anaconda3\lib\site-packages\pandas\core\frame.py:3509: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[k1] = value[k2]
